$$
 \textbf{Miguel Julio de Freitas - 142527}
$$

$$
 \textbf{Victor Antonio Bueno - 140849}
$$

$$
 \textbf{Yuri Akira Shiba - 139888}
$$





# **Projeto 02 - Simulação de Neurônios**

## **Modelagem** 



Este projeto tem como objetivo compreender o funcionamento de um conjunto de neurônios, mais especificamente, em uma rede onde um está acoplado a outro.

Para tal, utilizaremos o modelo Van der Pol de oscilador de relaxamento, este leva em consideração duas variáveis x (excitatória) e y (inibitória).

$$
\dfrac {dx}{dt} = 3x - x^3 + 2 - y + I + ρ
$$

$$
\dfrac {dy}{dt} = ε (α(1+tanh(x/β)) - y )
$$


> Nas equações acima, I se trata de um esímulo externo, ε é um pequeno número positivo e tanto α como β se tratam de parâmetros do problema.

Nesse cenário, os neurônios poderão oscilar entre dois estados: pulso e relaxamento.

Ainda, a fim de realizar nosso estudo, iremos utilizar uma equação para descrever a forma de acoplamento entre esses neurônios e ela vai se dar por:

$$
Sᵢ = \sum\limits_{k∈N(i)}^{}wᵢₖH(xₖ - θ)
$$

> Onde Wik define a força deste acoplamento, N(i) se trata da vizinhança do neurônio i e H() define que se x for maior que theta o retorno é 1 e caso contrário 0, vale ressaltar que theta se trata de um limiar de corte.





Para este projeto, algumas características foram pré-definidas:
- A rede de neurônios acoplados formará uma rede regular circular.
- Haverão 500 neurônios nesta simulação.
- O grau de vizinhança dos neurônios será 2.

Além disso, teremos como parâmetros iniciais:

In [1]:
# Parâmetros do Modelo
nNeuronios = 500
theta = 0.5
alpha = 6.0
epsilon = 0.02
beta = 0.1
IAtivo = 0.2
IInativo = -0.02
wki = 0.1
rho = 0
deltaT = 0.01
tempoSimulacao = 10


## **Implementação**

A seguir as biliotecas utilizadas:

In [2]:
import random as rand # Para gerarmos valores aleatórios 
import matplotlib.pyplot as plt # Para plotarmos os gráficos

import math # Para utilização do método tanh()

O primeiro a ser feito será a criação de métodos para auxiliar os cálculos.

In [3]:
def calculaAcoplamento(xAnterior, xProximo):
  if (xAnterior >= theta and xProximo >= theta):
    return 2 * wki
  else:
    if (xAnterior >= theta or xProximo >= theta):
      return wki

  return 0

def calculaDeltaX(neuronio, xAnterior, xProximo):
  indiceMaxX = len(neuronio.x) - 1
  indiceMaxY = len(neuronio.y) - 1
  dX = (
    3 * neuronio.x[indiceMaxX] - (neuronio.x[indiceMaxX] ** 3) + 2 - neuronio.y[indiceMaxY] + neuronio.I + rho + calculaAcoplamento(xAnterior, xProximo)
  ) * deltaT

  neuronio.x.append(neuronio.x[indiceMaxX] + dX)

def calculaDeltaY(neuronio):
  indiceMaxX = len(neuronio.x) - 2
  indiceMaxY = len(neuronio.y) - 1
  dY = epsilon * (alpha * (1 + math.tanh(neuronio.x[indiceMaxX] / beta )) - neuronio.y[indiceMaxY]) * deltaT

  neuronio.y.append(neuronio.y[indiceMaxY] + dY)

Aqui temos a definição de uma estrutura simples para os neurônios:

In [4]:
class Neuronio:
    def __init__(self, I):
        self.x = [rand.random() * 4 - 2]
        self.y = [rand.random() * 4]
        self.I = I
        #self.I = IAtivo if rand.randrange(10) >= 5 else IInativo

Agora, podemos fazer o método que criará a rede circular regular de grau 2 com os 500 neurônios.

In [5]:
'''
redeNeuronios = []

for i in range(nNeuronios):
    redeNeuronios.append(Neuronio())
'''

def iniciaRede(nNeuronios, redeNeuronios, nAtivos):
  for i in range(nNeuronios):
    if (i <= nAtivos):
      redeNeuronios.append(Neuronio(IAtivo))
    else:
      redeNeuronios.append(Neuronio(IInativo))

  return redeNeuronios

E então definimos a realização da simulação:

In [6]:
'''
for i in range(tempoSimulacao):
    for j in range(nNeuronios):
        ant = redeNeuronios[(j - 1) % nNeuronios]
        xDoAnterior = ant.x[len(ant.x) - 1]

        prox = redeNeuronios[(j + 1) % nNeuronios]
        xDoProximo = prox.x[len(prox.x) - 1]

        calculaDeltaX(redeNeuronios[j], xDoAnterior, xDoProximo)
        calculaDeltaY(redeNeuronios[j])
'''

def realizaSimulacao(nNeuronios, redeNeuronios, tempoSimulacao,intervaloDeDados):
  x = []
  y = []
  
  for i in range(tempoSimulacao):
    if (i % intervaloDeDados == 0):
      x.append([])
      y.append([])
    for j in range(nNeuronios):
      ant = redeNeuronios[(j - 1) % nNeuronios]
      xDoAnterior = ant.x[len(ant.x) - 1]

      prox = redeNeuronios[(j + 1) % nNeuronios]
      xDoProximo = prox.x[len(prox.x) - 1]

      calculaDeltaX(redeNeuronios[j], xDoAnterior, xDoProximo)
      calculaDeltaY(redeNeuronios[j])
      if (i % intervaloDeDados == 0):
        x[i // intervaloDeDados].append(redeNeuronios[j].x)
        y[i // intervaloDeDados].append(redeNeuronios[j].y)

            #descobrir pq isso da errado para q possamos armazenar apenas alguns valores entre intervalos de tempo
            #x[i].append(redeNeuronios[j].x)
            #y[i].append(redeNeuronios[j].y)

  return x, y

## **Gráficos**

### Simulação com todos os neurônios ativos

In [ ]:
redeNeuronios = []
nNeuronios = 100
tempoSimulacao = 500000
intervaloDeDados = 1000
redeNeuronios = iniciaRede(nNeuronios, redeNeuronios, nNeuronios)
#dados = realizaSimulacao(nNeuronios, redeNeuronios, tempoSimulacao, intervaloDeDados)
realizaSimulacao(nNeuronios, redeNeuronios, tempoSimulacao, intervaloDeDados)

for i in range(nNeuronios):
  plt.plot(range(tempoSimulacao + 1) ,redeNeuronios[i].y )
  #plt.plot(range(tempoSimulacao // intervaloDeDados + 1), dados[1][i] )

plt.show()

### Simulação com metade dos neurônios inativos

In [ ]:
redeNeuronios = []
nNeuronios = 100
nNeuroniosAtivos = (nNeuronios/2) - 1
tempoSimulacao = 200000
intervaloDeDados = 1

redeNeuronios = iniciaRede(nNeuronios, redeNeuronios, 10)
realizaSimulacao(nNeuronios, redeNeuronios, tempoSimulacao, intervaloDeDados)


for i in range(nNeuronios):
  plt.plot(range(tempoSimulacao + 1) ,redeNeuronios[i].y )

#plt.figure(figsize=(100,9))
#plt.show()


plt.show()

### Simulação com todos os neurônios inativos

In [ ]:
redeNeuronios = []
nNeuronios = 100
nNeuroniosAtivos = 0 - 1
tempoSimulacao = 100000
intervaloDeDados = 1

redeNeuronios = iniciaRede(nNeuronios, redeNeuronios, nNeuroniosAtivos)
realizaSimulacao(nNeuronios, redeNeuronios, tempoSimulacao, intervaloDeDados)


for i in range(nNeuronios):
  plt.plot(range(tempoSimulacao + 1) ,redeNeuronios[i].y )

#plt.figure(figsize=(100,9))
#plt.show()


plt.show()

### Alterando força de acoplamento (wki)

In [ ]:
wki = 0.0

In [ ]:
wki = 1.0

## **Conclusões**

Atráves da simulação e dos gráficos obtidos com ela podemos notar alguns fatos:

- Os neurônios, se incialmente estiverem todos ativos, terão a tendência de sempre se sincronizar. Com a variação dos fatores pode demorar mais ou menos tempo, mas isso ocorrerá em algum ponto.

- A quantidade de neurônios ativos é um desses fatores citados acima, quanto mais neurônios inativos, maior é a pertubação da rede dificultando a sincronização.

- Como é de se imaginar, o acoplamento é essencial para que os neurônios se sincronizem. Se não houver acoplamento cada neurônio manterá seu estado inicial sem ser influenciado pelos outros a sua volta. 

TENTAR RESPODER:

*   Qual o tempo para se atingir a sincronização?

*   A força de acoplamento importa? **RESPONDIDO**


*  Qual o percentual mínimo de neurônios ativos para sincronização?.